# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Содержание
1. [Откройте и изучите файл](#0),
2. [Разбейте данные на выборки](#1),
3. [Исследуйте модели](#2),
    * [Дерево решений](#5),
    * [Случайный лес](#6),
    * [Логистическая регрессия](#7),
4. [Проверьте модель на тестовой выборке](#3),
5. [Проверьте модели на адекватность](#4),

## 1. Откройте и изучите файл <a id='0'></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Датасет состоит из 3214 объектов и 5 признаков. Целевой признак для задачи - is_ultra, модель будет предсказывать значение 1 если клиенту нужно предложить тариф 'Ультра' или 0 если тариф 'Смарт'.

## 2. Разбейте данные на выборки <a id='1'></a>

Исходный набор данных разобью на три части: обучающую, валидационную и тестовую.

In [4]:
#60% под обучающую выборку
df_train, df_valid = train_test_split(df, test_size = 0.4, random_state = 12345)
df_valid, df_test = train_test_split(df_valid, test_size = 0.5, random_state = 12345)

В итоге получил следующее соотношение: 60%|20%|20%.

## 3. Исследуйте модели <a id='2'></a>

Для решения задачи классификации рассмотрю следующие изученные модели:
* дерево решений
* случайный лес
* логистическую регрессию

In [5]:
#целевые признаки обучающей и валидационных выборок
features_train = df_train.drop(['is_ultra'], axis = 1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis = 1)
target_valid = df_valid['is_ultra']

In [6]:
#функция рассчитывающая оценку правильности на валидационной выборке для обученной модели
def accuracy_score_valid(model):
    predictions_valid = model.predict(features_valid)
    return accuracy_score(predictions_valid, target_valid)

## Дерево решений <a id='5'></a>

In [7]:
for max_depth in range(1, 30, 2):
    model = DecisionTreeClassifier(max_depth = max_depth, random_state = 12345)
    model.fit(features_train, target_train)
    print(f'max_depth = {max_depth}:', accuracy_score_valid(model))

max_depth = 1: 0.7542768273716952
max_depth = 3: 0.7853810264385692
max_depth = 5: 0.7791601866251944
max_depth = 7: 0.7822706065318819
max_depth = 9: 0.7822706065318819
max_depth = 11: 0.7620528771384136
max_depth = 13: 0.7558320373250389
max_depth = 15: 0.7465007776049767
max_depth = 17: 0.7356143079315708
max_depth = 19: 0.7278382581648523
max_depth = 21: 0.7278382581648523
max_depth = 23: 0.7169517884914464
max_depth = 25: 0.713841368584759
max_depth = 27: 0.713841368584759
max_depth = 29: 0.713841368584759


Дерево решений имеет самую высокую оценку правильности 0.7853810264385692, когда задан гиперпараметр глубины равный 3.

## Случайный лес <a id='6'></a>

In [8]:
for estim in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators = estim, max_depth = 3, random_state = 12345)
    model.fit(features_train, target_train)
    print(f'n_estimators = {estim}:', accuracy_score_valid(model))

n_estimators = 10: 0.7853810264385692
n_estimators = 20: 0.7869362363919129
n_estimators = 30: 0.7869362363919129
n_estimators = 40: 0.7869362363919129
n_estimators = 50: 0.7869362363919129
n_estimators = 60: 0.7869362363919129
n_estimators = 70: 0.7884914463452566
n_estimators = 80: 0.7884914463452566
n_estimators = 90: 0.7884914463452566
n_estimators = 100: 0.7884914463452566


Модель случайного леса предсказывает тариф немного точнее при количестве деревьев 70 - 0.7884914463452566

## Логистическая регрессия <a id='7'></a>

In [9]:
def logistic_regression(solver):
    model = LogisticRegression(solver = solver, random_state = 12345)
    model.fit(features_train, target_train)
    return accuracy_score_valid(model)

In [10]:
logistic_regression('liblinear')

0.7589424572317263

In [11]:
import warnings
warnings.filterwarnings('ignore')

logistic_regression('newton-cg')

0.7558320373250389

In [12]:
logistic_regression('lbfgs')

0.7107309486780715

Даже используя другой алгоритм для решения задачи оптимизации модель логистической регрессии менее точна. Лучшей моделью получилась модель случайного леса.

## 4. Проверьте модель на тестовой выборке <a id='3'></a>

В результате наших экспериментов на валидационной выборке мы выяснили, что самую высокую оценку правильности дает лес решений с глубиной равной 3 и количеством деревьев равным 70.

In [13]:
features_test = df_test.drop(['is_ultra'], axis = 1)
target_test = df_test['is_ultra']

model = RandomForestClassifier(n_estimators = 70, max_depth = 3, random_state = 12345)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
accuracy_score(predictions_test, target_test)

0.7791601866251944

На тестовой выборке модель случайного леса показала приближенное к тренировачной значение accuracy.

## 5. (бонус) Проверьте модели на адекватность <a id='4'></a>

In [14]:
from sklearn.dummy import DummyClassifier
strategies = ['most_frequent', 'stratified', 'uniform', 'prior'] 
for name in strategies:
    dummy_model = DummyClassifier(random_state = 12345, strategy = name)
    dummy_model.fit(features_train, target_train)
    score = dummy_model.score(features_train, target_train)
    print('Для стратегии', name, 'средняя точность =', score)

Для стратегии most_frequent средняя точность = 0.6924273858921162
Для стратегии stratified средняя точность = 0.5980290456431535
Для стратегии uniform средняя точность = 0.49118257261410786
Для стратегии prior средняя точность = 0.6924273858921162


Проверив наши данныне используя DummyClassifier, мы видим, что значение accuracy у нашей модели значительно выше, чем у DummyClassifier. Из этого делаем вывод, что предсказания нашей модели адекватны.